# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_70301: Sweden, Finland, Norway, Denmark, Iceland<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("lastfm.csv")
data

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
...,...,...,...,...
289950,19718,bob dylan,f,Canada
289951,19718,pixies,f,Canada
289952,19718,the clash,f,Canada
289953,19718,a tribe called quest,f,Canada


1. Выбрать данные по странам своей группы (совместно): <br/>
3530203_70101: Germany, Netherlands <br/>

In [3]:
dat = data[data.country == ('Germany')]
dat = dat.append(data[data.country == ('Netherlands')], ignore_index=True)
dat

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
...,...,...,...,...
33919,19685,the cure,m,Netherlands
33920,19685,alice in chains,m,Netherlands
33921,19685,haggard,m,Netherlands
33922,19685,system of a down,m,Netherlands


In [4]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33924 entries, 0 to 33923
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     33924 non-null  int64 
 1   artist   33924 non-null  object
 2   sex      33924 non-null  object
 3   country  33924 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


In [5]:
dat_no_nd = dat[(dat['artist'] != '[unknown]')]
dat_no_nd = dat_no_nd.drop_duplicates()
dat_no_nd.count()

user       33828
artist     33828
sex        33828
country    33828
dtype: int64

2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.

In [6]:
#Объединим в группы по artist
dat_by_artist = dat_no_nd.groupby(['user', 'sex', 'country'])['artist'].apply(','.join).reset_index()
dat_by_artist.head()

,user,sex,country,artist
0,1,f,Germany,"red hot chili peppers,the black dahlia murder,..."
1,31,f,Netherlands,"jamiroquai,tori amos,jazzanova,radiohead,st. g..."
2,33,f,Germany,"death cab for cutie,tegan and sara,kimya dawso..."
3,42,f,Germany,"soundtrack,groove coverage,avril lavigne,the r..."
4,51,f,Germany,"kate nash,arctic monkeys,lykke li,adele,tegan ..."


In [7]:
#Сопоставляем artist с самими собой
dummy_dat_by_artist = dat_by_artist['artist'].str.get_dummies(',')
dat_art_art = pd.concat([dat_by_artist, dummy_dat_by_artist], axis = 1)

dat_art_art.head(5)

,user,sex,country,artist,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
0,1,f,Germany,"red hot chili peppers,the black dahlia murder,...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,31,f,Netherlands,"jamiroquai,tori amos,jazzanova,radiohead,st. g...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,33,f,Germany,"death cab for cutie,tegan and sara,kimya dawso...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,42,f,Germany,"soundtrack,groove coverage,avril lavigne,the r...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,51,f,Germany,"kate nash,arctic monkeys,lykke li,adele,tegan ...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#Находим поддержку с помошью apriori
from mlxtend.frequent_patterns import apriori, association_rules
apriori_support = apriori(dat_art_art.drop(columns=['user','sex','country','artist']), min_support=0.02, use_colnames=True)
apriori_support

,support,itemsets
0,0.057458,(3 doors down)
1,0.041207,(30 seconds to mars)
2,0.020313,(50 cent)
3,0.029019,(a perfect circle)
4,0.031921,(abba)
...,...,...
518,0.024376,"(system of a down, slipknot)"
519,0.028439,"(system of a down, the offspring)"
520,0.023796,"(the doors, the beatles)"
521,0.022055,"(the rolling stones, the beatles)"


In [9]:
#Находим правила о зависимости между artist и другими artist из apriori_support
rules_by_apriori = association_rules(apriori_support, metric="confidence", min_threshold=0.5)
rules_by_apriori.sort_values(by=['lift'], ascending=False)
#rules_by_apriori[rules_by_apriori['consequents'] == {'subway to sally'}].sort_values(by=['lift'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(schandmaul),(subway to sally),0.044689,0.055717,0.023796,0.532468,9.556683,0.021306,2.019717
3,(in extremo),(subway to sally),0.052815,0.055717,0.026698,0.505495,9.072573,0.023755,1.909551
2,(farin urlaub),(die Ärzte),0.058619,0.143355,0.035984,0.613861,4.282118,0.027580,2.218492
6,(the doors),(the beatles),0.047591,0.123622,0.023796,0.500000,4.044601,0.017912,1.752757
1,(die toten hosen),(die Ärzte),0.084736,0.143355,0.047591,0.561644,3.917864,0.035444,1.954222
4,(limp bizkit),(linkin park),0.052815,0.142194,0.027858,0.527473,3.709531,0.020348,1.815357
0,(keane),(coldplay),0.037725,0.171793,0.023796,0.630769,3.671674,0.017315,2.243060


In [10]:
#Сопоставляем artist с sex
dat_art_sex = pd.concat([dat_by_artist['sex'].str.get_dummies(','), dat_by_artist['artist'].str.get_dummies(',')], axis = 1)
dat_art_sex

,f,m,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1719,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1720,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1721,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#Находим поддержку с помошью fpgrowth
from mlxtend.frequent_patterns import fpgrowth
fpgrowth_support = fpgrowth(dat_art_sex, min_support=0.02, use_colnames=True)
fpgrowth_support

,support,itemsets
0,0.233894,(f)
1,0.154962,(red hot chili peppers)
2,0.105049,(the killers)
3,0.099826,(jack johnson)
4,0.048172,(the rolling stones)
...,...,...
823,0.023215,"(m, deep purple)"
824,0.027858,"(m, frank sinatra)"
825,0.028439,"(m, ensiferum)"
826,0.020313,"(m, katie melua)"


In [12]:
#Находим правила о зависимости между artist и sex из fpgrowth_support
rules_by_fpgrowth = association_rules(fpgrowth_support, metric="confidence", min_threshold=0.5)
rules_by_fpgrowth[rules_by_fpgrowth['consequents'] == {'m'}].append(rules_by_fpgrowth[rules_by_fpgrowth['consequents'] == {'f'}], ignore_index=True).sort_values(by=['lift'], ascending=False)
#rules_by_fpgrowth[rules_by_fpgrowth['consequents'] == {'f'}].sort_values(by=['lift'], ascending=False)
#rules_by_fpgrowth[rules_by_fpgrowth['consequents'] == {'m'}].sort_values(by=['lift'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
269,(paramore),(f),0.037145,0.233894,0.021474,0.578125,2.471735,0.012786,1.815954
271,(pink),(f),0.037725,0.233894,0.021474,0.569231,2.433709,0.012651,1.778459
270,(kate nash),(f),0.040627,0.233894,0.022055,0.542857,2.320950,0.012552,1.675856
188,(motörhead),(m),0.028439,0.766106,0.027278,0.959184,1.252025,0.005491,5.730412
108,(miles davis),(m),0.024376,0.766106,0.023215,0.952381,1.243146,0.004541,4.911782
...,...,...,...,...,...,...,...,...,...
39,(mando diao),(m),0.065583,0.766106,0.035984,0.548673,0.716184,-0.014260,0.518237
26,(the kooks),(m),0.094602,0.766106,0.051654,0.546012,0.712711,-0.020821,0.515200
232,(damien rice),(m),0.044689,0.766106,0.024376,0.545455,0.711983,-0.009861,0.514568
131,(my chemical romance),(m),0.041788,0.766106,0.022055,0.527778,0.688910,-0.009959,0.495306


In [13]:
#Сопоставляем artist с country
dat_art_cou = pd.concat([dat_by_artist['country'].str.get_dummies(','), dat_by_artist['artist'].str.get_dummies(',')], axis = 1)
dat_art_cou

,Germany,Netherlands,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,...,wilco,within temptation,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1718,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1719,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1720,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1721,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#Находим поддержку с помошью fpgrowth
from mlxtend.frequent_patterns import fpmax
fpmax_support = fpgrowth(dat_art_cou, min_support=0.02, use_colnames=True) #тут должен быть fpmax, но он выдает ошибку
fpmax_support

,support,itemsets
0,0.728961,(Germany)
1,0.154962,(red hot chili peppers)
2,0.105049,(the killers)
3,0.099826,(jack johnson)
4,0.048172,(the rolling stones)
...,...,...
869,0.021474,"(editors, Netherlands)"
870,0.022635,"(black sabbath, Germany)"
871,0.021474,"(james blunt, Germany)"
872,0.031341,"(ensiferum, Germany)"


In [15]:
#Находим правила о зависимости между artist и country из fpmax_support
rules_by_fpmax = association_rules(fpmax_support, metric="confidence", min_threshold=0.5)
rules_by_fpmax[rules_by_fpmax['consequents'] == {'Germany'}].append(rules_by_fpmax[rules_by_fpmax['consequents'] == {'Netherlands'}], ignore_index=True).sort_values(by=['lift'], ascending=False)
#rules_by_fpmax[rules_by_fpmax['consequents'] == {'Germany'}].sort_values(by=['lift'], ascending=False)
#rules_by_fpmax[rules_by_fpmax['consequents'] == {'Netherlands'}].sort_values(by=['lift'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
311,(jamiroquai),(Netherlands),0.042948,0.271039,0.022055,0.513514,1.894612,0.010414,1.498420
312,(mika),(Netherlands),0.040046,0.271039,0.020313,0.507246,1.871489,0.009459,1.479362
314,(editors),(Netherlands),0.042368,0.271039,0.021474,0.506849,1.870024,0.009991,1.478171
313,(u2),(Netherlands),0.056297,0.271039,0.028439,0.505155,1.863772,0.013180,1.473109
53,"(incubus, die Ärzte)",(Germany),0.022635,0.728961,0.022635,1.000000,1.371815,0.006135,inf
...,...,...,...,...,...,...,...,...,...
149,(röyksopp),(Germany),0.044689,0.728961,0.024956,0.558442,0.766079,-0.007620,0.613823
235,(moby),(Germany),0.059779,0.728961,0.033082,0.553398,0.759160,-0.010495,0.606891
277,(damien rice),(Germany),0.044689,0.728961,0.024376,0.545455,0.748263,-0.008201,0.596286
301,(led zeppelin),(Germany),0.039466,0.728961,0.021474,0.544118,0.746429,-0.007295,0.594537


3. Вывести эти правила в отдельных ячейках.

In [16]:
rules_by_apriori[rules_by_apriori['consequents'] == {'subway to sally'}].sort_values(by=['lift'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(schandmaul),(subway to sally),0.044689,0.055717,0.023796,0.532468,9.556683,0.021306,2.019717
3,(in extremo),(subway to sally),0.052815,0.055717,0.026698,0.505495,9.072573,0.023755,1.909551


In [17]:
rules_by_fpgrowth[rules_by_fpgrowth['consequents'] == {'f'}].sort_values(by=['lift'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
54,(paramore),(f),0.037145,0.233894,0.021474,0.578125,2.471735,0.012786,1.815954
132,(pink),(f),0.037725,0.233894,0.021474,0.569231,2.433709,0.012651,1.778459
76,(kate nash),(f),0.040627,0.233894,0.022055,0.542857,2.320950,0.012552,1.675856


In [18]:
rules_by_fpgrowth[rules_by_fpgrowth['consequents'] == {'m'}].sort_values(by=['lift']).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
196,(mika),(m),0.040046,0.766106,0.020313,0.507246,0.662110,-0.010366,0.474668
142,(my chemical romance),(m),0.041788,0.766106,0.022055,0.527778,0.688910,-0.009959,0.495306
246,(damien rice),(m),0.044689,0.766106,0.024376,0.545455,0.711983,-0.009861,0.514568
27,(the kooks),(m),0.094602,0.766106,0.051654,0.546012,0.712711,-0.020821,0.515200
42,(mando diao),(m),0.065583,0.766106,0.035984,0.548673,0.716184,-0.014260,0.518237


In [19]:
rules_by_fpmax[rules_by_fpmax['consequents'] == {'Germany'}].sort_values(by=['lift'], ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
101,"(die Ärzte, arctic monkeys)",(Germany),0.021474,0.728961,0.021474,1.0,1.371815,0.005820,inf
70,(farin urlaub),(Germany),0.058619,0.728961,0.058619,1.0,1.371815,0.015888,inf
275,"(die Ärzte, muse)",(Germany),0.020894,0.728961,0.020894,1.0,1.371815,0.005663,inf
43,"(die Ärzte, foo fighters)",(Germany),0.023796,0.728961,0.023796,1.0,1.371815,0.006450,inf
230,"(green day, die Ärzte)",(Germany),0.026117,0.728961,0.026117,1.0,1.371815,0.007079,inf


In [20]:
rules_by_fpmax[rules_by_fpmax['consequents'] == {'Netherlands'}].sort_values(by=['lift'], ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
24,(jamiroquai),(Netherlands),0.042948,0.271039,0.022055,0.513514,1.894612,0.010414,1.498420
255,(mika),(Netherlands),0.040046,0.271039,0.020313,0.507246,1.871489,0.009459,1.479362
326,(editors),(Netherlands),0.042368,0.271039,0.021474,0.506849,1.870024,0.009991,1.478171
281,(u2),(Netherlands),0.056297,0.271039,0.028439,0.505155,1.863772,0.013180,1.473109


4. Подумать, как можно было бы использовать полученные правила.

1) Для рекоммендаций на сервисах прослушивания музыки <br>
2) Для выбора групп на разогреве на концерте <br>
3) Для выбора стран в которыых группам стоит или не стоит продвигаться и устраивать концерты